<a href="https://colab.research.google.com/github/andjoer/llm_poetry_generation/blob/main/colabs/GPT2_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Texterzeugung mit GPT2
Unter verwendung der Huggingface Bibliothek

In [ ]:
!pip install transformers

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import os
import pandas as pd
import numpy as np

import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('Anjoe/german-poetry-gpt2')

Der Tokenizer wandelt Worte oder Teile von Worten in Zahlen um. Dabei ist die Zuordnung fix in einem Wörterbuch gespeichert. Diese Zuordnungstabelle hat für das hier verwendete Modell 50265 Einträge

In [16]:
len(tokenizer)

50265

Das Wort Haus ist so häufig, dass es eine eigene Nummer bekommt

In [4]:
tokenizer.encode('Haus')

[6699]

Ein Leerzeichen wird zum nachfolgenden Wort hinzugerechnet, sodass " Haus" eine andere Nummer bekommt wie "Haus"

In [5]:
tokenizer.encode(' Haus')

[1046]

Das Wort "Brühe" besteht aus zwei Token. Indem man diese Token decodiert kann man erkennen, wie das Wort zerlegt wurde

In [19]:
tokenizer.encode('Brühe')

[15731, 462]

In [20]:
tokenizer.decode(15731)

'Brü'

Jedem Wort, auch unbekannten Worten, kann eine Nummer zugewiesen werden, da auch die einzelnen Buchstaben Nummern haben. Ein unbekanntes Wort würde also in sehr viele Teile zerlegt werden, ist jedoch für GPT2 lesbar 

In [8]:
tokenizer.encode('r')

[86]

In [9]:
model = GPT2LMHeadModel.from_pretrained('Anjoe/german-poetry-gpt2')  # or any other checkpoint
model.eval()
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 


def encode_word(text):
    text_index = tokenizer.encode(text,add_prefix_space=True)
    vector = model.transformer.wte.weight[text_index,:]    
    return vector

vector_1 = encode_word('Haus')


Downloading:   0%|          | 0.00/955 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Das Zuweisen von Nummern durch den Tokenizer ist der erste notwendige Schritt, da ein Algorithmus nur numerische Werte versteht. Diese Token tragen jedoch noch keine Bedeutung. Die erste Ebene des Sprachmodells lernt, aus die bedeutungslosen Zahlen in Vektoren zu übersetzen, welche die Bedeutung repräsentieren. Im Falle des verwendeten Modells hat der Vektor 768 Dimensionen.

In [21]:
print(vector_1.shape)
print(vector_1)

torch.Size([1, 768])
tensor([[-1.3986e-01,  9.3552e-02,  1.7866e-01,  1.0349e-02, -4.2736e-01,
         -1.9803e-01,  2.9587e-01,  1.4574e-01, -3.6000e-01, -1.6051e-01,
          2.0360e-01,  2.7261e-01, -3.2524e-01,  5.0175e-02, -8.2143e-02,
         -2.1913e-01,  8.2726e-02, -2.2122e-02, -1.9095e-01,  1.0004e-01,
         -2.6142e-02,  6.7242e-02, -3.1882e-02, -2.0613e-01, -1.0150e-01,
         -2.7248e-01, -1.9087e-01, -1.8369e-02, -1.2854e-01,  1.9363e-02,
         -4.6236e-01, -5.4354e-02, -1.0212e-01, -1.6742e-02, -1.6608e-01,
         -1.6354e-01,  9.6244e-02,  3.6504e-01, -3.5425e-01, -9.8744e-02,
         -1.6871e-01,  2.0984e-03,  2.9146e-01, -6.8037e-02,  1.6037e-01,
         -1.1593e-01, -5.0723e-02, -2.0560e-01,  1.4856e-01, -2.6181e-01,
          1.5041e-01, -2.9828e-01,  1.3498e-01,  7.6111e-02, -1.3833e-03,
         -8.4609e-04,  5.4982e-02,  8.4038e-02,  5.2760e-02,  4.3140e-02,
          1.3263e-02, -1.9020e-01, -1.3285e-01, -9.6504e-02, -6.5043e-02,
         -2.5697e

Das Sprachmodell berechnet nun für eine bestimmte Prompt die Wahrscheinlichkeit für das nachfolende Token. Hierbei ist zu beachten, dass jedes Token aus dem oben genannten Wörterbuch eine Wahrscheinlichkeit zugewiesen bekommt. Das Modell gibt also 50265 verschiedene Wahrscheinlichkeiten aus. Im nachfolgenden Beispiel ist die Prompt das Wort "Ich". Die angezeigten Token werden ihrer Wahrscheinlichkeit nach sortiert. 

In [10]:
text = '''Ich'''

inputs = tokenizer(text,return_tensors='pt')['input_ids']

outputs = model(inputs)

probabilities = outputs.logits[:,-1,:] 

next_token_sample = torch.argsort(-probabilities)
    
next_token_sample[0]

tensor([ 2399,  1251,  1145,  ..., 27954, 26694, 17306])

Unten stehend werden die 10 wahrscheinlichsten Token-IDs *decodiert*. 

In [15]:
for i in range(10):
  print(tokenizer.decode(next_token_sample[0][i]))

 hab
 bin
 will
 kann
 war
 werd
 weiß
 habe
 rief
 w


Das unwahrscheinlichste Token ist "irur"

In [11]:
tokenizer.decode(17306)

'irur'

Mithilfe einer for-Schleife wird nun ein Text erzeugt, wobei immer das wahrscheinlichste Token zur Fortsetzung des Textes verwendet wird. Das Resultat ist bei jedem Versuch das exakt gleiche, da das Modell selbst deterministisch ist. Somit überrascht es nicht, dass auf das "Ich" das "hab" folgt, da dies bereits die Vorhersage für das wahrscheinlichste Token im letzten Versuch war. 

In [ ]:
text = '''Ich'''

inputs = tokenizer(text,return_tensors='pt')['input_ids']

for i in range(10):
    outputs = model(inputs)
    
    probabilities = outputs.logits[:,-1,:] 
  
    next_token_sample = torch.argsort(-probabilities)

    next_token = next_token_sample[:,0]
    
    inputs = torch.cat((inputs, torch.reshape(next_token,(1,1))),1)

print(tokenizer.decode(inputs[0]))

Ich hab es nicht vergessen,
Und ich hab es


Man kann jedoch aus den wahrscheinlichsten Token mithilfe eines Zufallsgenerator eines auswählen. Auf diese Weise verhindert man mit großer Wahrscheinlichkeit Plagiate. Umso größer der Bereich ist, aus dem man wählt, desto kreativer werden die Ausgaben - jedoch können sie dadurch auch zunehmend sinnlos werden. Das Sampeln aus der Wahrscheinlichkeitsverteilung ermöglicht auf jeden Fall interessante poetische Gesten. Man könnte zum Beispiel auf einem bestimmten Trainingskorpus trainieren und bei der Erzeugung immer das zehnt wahrscheinlichste Token verwenden. 

In [ ]:
text = '''Ich'''

inputs = tokenizer(text,return_tensors='pt')['input_ids']

for i in range(10):
    outputs = model(inputs)
    
    probabilities = outputs.logits[:,-1,:] 
  
    next_token_sample = torch.argsort(-probabilities)
    
    random = np.random.randint(10)
    
    next_token = next_token_sample[:,random]
    
    inputs = torch.cat((inputs, torch.reshape(next_token,(1,1))),1)

print(tokenizer.decode(inputs[0]))

Ich weiß, sie sind die Mutter
Der Unschuld,
